In [1]:
# pip install konlpy

In [2]:
# !pip install soyspacing

In [3]:
# pip install soynlp

In [4]:
# !pip install graphviz

In [5]:
# !pip install fsspec

In [6]:
# pip install --user JPype1-1.2.0-cp38-cp38-win_amd64.whl

In [ ]:
pip install sys


In [ ]:
import sys
sys.version

In [8]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from konlpy.tag import Hannanum
from konlpy.tag import Komoran
from hanspell import spell_checker
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
from sklearn.linear_model import Lasso
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

import numpy as np
import seaborn as sns
import tensorflow as tf
import os
import graphviz
from tqdm.notebook import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc, roc_curve
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
from soynlp.normalizer import *
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

In [9]:
df = pd.read_csv('./2021-03-24_17-46-26 티빙.csv')
df.head()

,name,ratings,date,helpful,comment,developer_comment
0,Ru,1,2021-03-24,0,돈 안 받고 쓰는 무료어플이냐고욬ㅋ 아니 다른 앱들도 돈 내고 여러개 쓰는데 이정도...,NaN
1,Google 사용자,1,2021-03-24,0,아니 끝까지 보려면 로그인을 꼭 해야하나요? 무료보기는 꼴랑3분!!!,씨제이이엔엠(CJ ENM)2021년 3월 24일안녕하세요. TVING입니다. TVI...
2,한ᄒᄅ,1,2021-03-24,0,한글자막없다!!!한국드라마보고싶는데 자막없네요..ㅡㅡ,씨제이이엔엠(CJ ENM)2021년 3월 24일안녕하세요. TVING 고객센터입니다...
3,Hyeonjeong Lim,1,2021-03-24,0,앱처음화면에서 로딩이 넘어가지않아 재설치후 로그인하려니 네이버버튼 로그인 누르니 그...,NaN
4,김진영,1,2021-03-23,0,로딩문제,NaN


In [10]:
df.describe() # data 살펴보기

,ratings,helpful
count,15957.000000,15957.000000
mean,2.620731,1.269161
std,1.695863,6.878949
min,1.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.000000
75%,5.000000,0.000000
max,5.000000,193.000000


In [11]:
df = df[['date', 'comment', 'ratings']] # 날짜, 리뷰내용, 평점

In [12]:
df = df.dropna(axis = 0, how = 'any') # 모든 행에서 NaN 값이 하나라도 있으면 삭제(결측치 제거)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15925 entries, 0 to 15956
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     15925 non-null  object
 1   comment  15925 non-null  object
 2   ratings  15925 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 497.7+ KB


In [13]:
df['comment'] # comment 확인

0        돈 안 받고 쓰는 무료어플이냐고욬ㅋ 아니 다른 앱들도 돈 내고 여러개 쓰는데 이정도...
1                   아니 끝까지 보려면 로그인을 꼭 해야하나요? 무료보기는 꼴랑3분!!!
2                            한글자막없다!!!한국드라마보고싶는데 자막없네요..ㅡㅡ
3        앱처음화면에서 로딩이 넘어가지않아 재설치후 로그인하려니 네이버버튼 로그인 누르니 그...
4                                                     로딩문제
                               ...                        
15952     디자이어 아주 잘됩니다. 더좋아지겠죠? 볼만한건 유료라지만 유료라도이정도면 볼만한듯..
15953                           세상에공짜는 없다 볼만한건전부 유료다. 바로삭제
15954                                           갤S인데 오류쩔어요
15955                   채널수와 화질에서는 DMB와는 비교가 안되는군요 티빙짱입니다!
15956                               우와! 이론 멋진 앱이 있다니.띠용~^^
Name: comment, Length: 15925, dtype: object

In [14]:
from soyspacing.countbase import CountSpace
def 띄어쓰기(text):
    verbose=False
    mc = 10  # min_count
    ft = 0.3 # force_abs_threshold
    nt =-0.3 # nonspace_threshold
    st = 0.3 # space_threshold


    sent_corrected, tags = model.correct(
    doc=text,
    verbose=verbose,
    force_abs_threshold=ft,
    nonspace_threshold=nt,
    space_threshold=st,
    min_count=mc)
    sent_corrected, tags = model.correct(text)
    result = sent_corrected
    return result
corpus_fname = '99714_norm.txt' # 첨부파일을 경로에 저장
model = CountSpace()
model.train(corpus_fname)    

df['comment'] = df['comment'].apply(lambda x: 띄어쓰기(x))
df.head()

all tags length = 477994 --> 46185, (num_doc = 13813)

,date,comment,ratings
0,2021-03-24,돈 안 받고 쓰는 무료어플이냐고욬ㅋ 아니 다른 앱들도 돈 내고 여러개 쓰는데 이정도...,1
1,2021-03-24,아니 끝까지 보려면 로그인을 꼭 해야하나요? 무료보기는 꼴랑3분!!!,1
2,2021-03-24,한글자막없다!!!한국드라마보고싶는데 자막없네요..ㅡㅡ,1
3,2021-03-24,앱처음화면에서 로딩이 넘어가지않아 재설치후 로그인하려니 네이버버튼 로그인 누르니 그...,1
4,2021-03-23,로딩문제,1


In [ ]:
#맞춤법 검사기
for i in tqdm(range(len(df))): 
    try: 
        result = spell_checker.check(df['comment'][i])
        a= result[1]
        df['comment'][i] = a
    except: 
        pass

  0%|          | 0/15925 [00:00<?, ?it/s]

<ipython-input-15-758cfaf193a0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'][i] = a


In [ ]:
def text_cleaning(text):
    result = only_hangle(text)
    result = repeat_normalize(result, num_repeats=3) #반복적인 단어 응축(3번 이상인것들)
    result = emoticon_normalize(result,num_repeats=3) #반복적인 자음,모음 응축(3번 이상인것들)
    return result
df['ko_text'] = df['comment'].apply(lambda x: text_cleaning(x))
df.head()

In [ ]:
from soynlp.word import WordExtractor
from soynlp.utils import DoublespaceLineCorpus
from soynlp.tokenizer import LTokenizer
word_extractor = WordExtractor(min_frequency=100,
   min_cohesion_forward=0.05,
   min_right_branching_entropy=0.0
)
word_extractor.train(df['ko_text'].values)
words = word_extractor.extract()
words

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)


In [ ]:
df = pd.read_csv('./티빙전처리.csv')
del df['Unnamed: 0']
df.head()

In [ ]:
df['nouns'] = df['ko_text'].apply(lambda x: tokenizer.tokenize(x, remove_r=True)) # L과 R 분리하여 R은 삭제
df

In [ ]:
df.describe() # 정제된 data 살펴보기

In [ ]:
df = df.reset_index() # 인덱스 제거
del df['index']

In [ ]:
# 한국어 약식 불용어사전 예시 파일입니다. 출처 - (https://www.ranks.nl/stopwords/korean)
korean_stopwords_path = "99714_norm.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    
    # 한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]
    
    # 불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    return nouns

In [ ]:
df

In [ ]:
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x)) # 키워드 뽑아내고, 불필요한 행 제거하기/토크나이저
for i in range(len(df)):
    if not df['nouns'][i]:
        df.drop([i],inplace=True)
        
df.reset_index(inplace=True)
del df['index']
print(df.shape)
df.head()

In [ ]:
all_words = []
satisfaction= []
for i in range(len(df)):
    words = df['nouns'][i]
    words_arr = []
    for n in words:
        words_arr.append(n)
        satisfaction.append(df['ratings'].iloc[i])
    all_words.extend(words_arr)
    
all_words_df = pd.DataFrame({"words":all_words, "satisfaction":satisfaction, "count":len(all_words)*[1]})
all_words_df.head()  

# EDA

In [ ]:
words_satisfaction = all_words_df.groupby('words').mean()['satisfaction']
words_count = all_words_df.groupby('words').sum()['count']
words_df = pd.concat([words_satisfaction, words_count], axis=1)
words_df.head(20)

In [ ]:
words_df = words_df.loc[words_df['count']>=3]
words_df.sort_values('satisfaction', ascending=False)

In [ ]:
words_df.sort_values('count', ascending =False).head(20)

In [ ]:
korean_stopwords_path = "./korean_stopwords3.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
nouns_tagger =Kkma()
def text_cleaning(text):
    result = only_hangle(text)
    result = repeat_normalize(result, num_repeats=2) #num_repeats 
    result = emoticon_normalize(result,num_repeats=2 )
    nouns_tagger = tokenizer
    nouns = nouns_tagger.tokenize(result, remove_r=True)
    nouns = ','.join(nouns)
    nouns_tagger2 =  Okt()
    nouns = nouns_tagger2.nouns(nouns)
    nouns = [x for x in nouns if len(x) >1]
    nouns =[x for x in nouns if x not in stopwords]
    return nouns

vect = CountVectorizer(tokenizer= lambda x : text_cleaning(x))
bow_vect = vect.fit_transform(df['comment'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [ ]:
bow_vect.shape

In [ ]:
word_count_dict = dict(zip(word_list, count_list))
print(str(word_count_dict)[:100])

In [ ]:
tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [ ]:
print(tf_idf_vect[0])

In [ ]:
invert_index_vectorizer = {v:k for k,v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100])

In [ ]:
df.sample(10).head()

In [ ]:
df.ratings.hist()

In [ ]:
def rating_to_label(rating) :
  if rating >3:
    return 1
  else:
    return 0
df['y'] = df['ratings'].apply(lambda x : rating_to_label(x))

In [ ]:
df

In [ ]:
# df.to_csv('./티빙전처리.csv') # 'sadsad.csv' 원하는 이름 작성 주의: 한번적하면 주석 또는 지울것 
# df = pd.read_csv('./티빙전처리.csv') 
# del df['Unnamed: 0']
# df

In [ ]:
df.y.value_counts()

In [ ]:
y= df['y']
X_train, X_test, y_train, y_test = train_test_split(tf_idf_vect, y, test_size = 0.2)

In [ ]:
print(X_test.shape)

# 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver="lbfgs", max_iter=500)
lr.fit(X_train, y_train)
print("Test Accuracy: {}%".format(round(lr.score(X_test, y_test)*100,2)))

In [ ]:
scores = cross_val_score(lr, tf_idf_vect, y, cv=10) # model, train, target, cross validation

print('cross-val-score \n{}'.format(scores))

print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))

In [ ]:
y_pred_lr = lr.predict(X_test)
print("Logistic Regression Classifier report: \n\n", classification_report(y_test, y_pred_lr))

In [ ]:
cm = confusion_matrix(y_test, y_pred_lr)
x_axis_labels = ["Edible", "Poisonous"]
y_axis_labels = ["Edible", "Poisonous"]
f, ax = plt.subplots(figsize =(7,7))
sns.heatmap(cm, annot = True, linewidths=0.2, linecolor="black", fmt = ".0f", ax=ax, cmap="Purples", xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.xlabel("PREDICTED LABEL")
plt.ylabel("TRUE LABEL")
plt.title('Confusion Matrix for Logistic Regression Classifier')
plt.savefig("lrcm.png", format='png', dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]
plt.bar(range(len(lr.coef_[0])), lr.coef_[0])

In [ ]:
coef_pos_index = sorted(((value, index ) for index, value in enumerate(lr.coef_[0])), reverse=True)
coef_neg_index = sorted(((value,index) for index, value in enumerate(lr.coef_[0])), reverse= False)

In [ ]:
for coef in coef_pos_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0]) #띄어쓰기 ,soynlp konlpy  한국어 

In [ ]:
for coef in coef_neg_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0])

# 의사결정 트리

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
y_pred_dt = dt.predict(X_test)
print("Decision Tree Classifier report: \n\n", classification_report(y_test, y_pred_dt))
print("Test Accuracy: {}%".format(round(dt.score(X_test, y_test)*100, 2)))

In [ ]:
cm = confusion_matrix(y_test, y_pred_dt)
x_axis_labels = ["Edible", "Poisonous"]
y_axis_labels = ["Edible", "Poisonous"]
f, ax = plt.subplots(figsize =(7,7))
sns.heatmap(cm, annot = True, linewidths=0.2, linecolor="black", fmt = ".0f", ax=ax, cmap="Purples", xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.xlabel("PREDICTED LABEL")
plt.ylabel("TRUE LABEL")
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.savefig("dtcm.png", format='png', dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]
plt.bar(range(len(dt.feature_importances_)), dt.feature_importances_)

In [ ]:
coef_pos_index = sorted(((value, index ) for index, value in enumerate(dt.feature_importances_)), reverse=True)
coef_neg_index = sorted(((value,index) for index, value in enumerate(dt.feature_importances_)), reverse= False)

In [ ]:
for coef in coef_pos_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0]) #띄어쓰기 ,soynlp konlpy  한국어 

In [ ]:
for coef in coef_neg_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0])

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
best_Kvalue = 0
best_score = 0
for i in range(2,10):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    if knn.score(X_test, y_test) > best_score:
        best_score = knn.score(X_train, y_train)
        best_Kvalue = i
        
print("Best KNN Value: {}".format(best_Kvalue))
print("Test Accuracy: {}%".format(round(best_score*100,2)))

In [ ]:
y_pred_knn = knn.predict(X_test)
print("KNN Classifier report: \n\n", classification_report(y_test, y_pred_knn))

In [ ]:
cm = confusion_matrix(y_test, y_pred_knn)
x_axis_labels = ["Edible", "Poisonous"]
y_axis_labels = ["Edible", "Poisonous"]
f, ax = plt.subplots(figsize =(7,7))
sns.heatmap(cm, annot = True, linewidths=0.2, linecolor="black", fmt = ".0f", ax=ax, cmap="Purples", xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.xlabel("PREDICTED LABEL")
plt.ylabel("TRUE LABEL")
plt.title('Confusion Matrix for KNN Classifier')
plt.savefig("knncm.png", format='png', dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]
plt.bar(range(len(dt.feature_importances_)), dt.feature_importances_)

In [ ]:
coef_pos_index = sorted(((value, index ) for index, value in enumerate(dt.feature_importances_)), reverse=True)
coef_neg_index = sorted(((value,index) for index, value in enumerate(dt.feature_importances_)), reverse= False)

In [ ]:
for coef in coef_pos_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0]) #띄어쓰기 ,soynlp konlpy  한국어 

In [ ]:
for coef in coef_neg_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0])

# SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC(random_state=42, gamma="auto")
svm.fit(X_train, y_train)
print("Test Accuracy: {}%".format(round(svm.score(X_test, y_test)*100, 2)))

In [ ]:
y_pred_svm = svm.predict(X_test)
print("SVM Classifier report: \n\n", classification_report(y_test, y_pred_svm))

In [ ]:
cm = confusion_matrix(y_test, y_pred_svm)
x_axis_labels = ["Edible", "Poisonous"]
y_axis_labels = ["Edible", "Poisonous"]
f, ax = plt.subplots(figsize =(7,7))
sns.heatmap(cm, annot = True, linewidths=0.2, linecolor="black", fmt = ".0f", ax=ax, cmap="Purples", xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.xlabel("PREDICTED LABEL")
plt.ylabel("TRUE LABEL")
plt.title('Confusion Matrix for SVM Classifier')
#plt.savefig("svmcm.png", format='png', dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]
plt.bar(range(len(dt.feature_importances_)), dt.feature_importances_)

In [ ]:
coef_pos_index = sorted(((value, index ) for index, value in enumerate(dt.feature_importances_)), reverse=True)
coef_neg_index = sorted(((value,index) for index, value in enumerate(dt.feature_importances_)), reverse= False)

In [ ]:
for coef in coef_pos_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0]) #띄어쓰기 ,soynlp konlpy  한국어 

In [ ]:
for coef in coef_neg_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0])

# 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print("Test Accuracy: {}%".format(round(rf.score(X_test, y_test)*100, 2)))

In [ ]:
y_pred_rf = rf.predict(X_test)
print("Random Forest Classifier report: \n\n", classification_report(y_test, y_pred_rf))

In [ ]:
cm = confusion_matrix(y_test, y_pred_rf)
x_axis_labels = ["Edible", "Poisonous"]
y_axis_labels = ["Edible", "Poisonous"]
f, ax = plt.subplots(figsize =(7,7))
sns.heatmap(cm, annot = True, linewidths=0.2, linecolor="black", fmt = ".0f", ax=ax, cmap="Purples", xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.xlabel("PREDICTED LABEL")
plt.ylabel("TRUE LABEL")
plt.title('Confusion Matrix for Random Forest Classifier');
plt.savefig("rfcm.png", format='png', dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]
plt.bar(range(len(rf.feature_importances_)), rf.feature_importances_)

In [ ]:
coef_pos_index = sorted(((value, index ) for index, value in enumerate(rf.feature_importances_)), reverse=True)
coef_neg_index = sorted(((value,index) for index, value in enumerate(rf.feature_importances_)), reverse= False)

In [ ]:
for coef in coef_pos_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0]) #띄어쓰기 ,soynlp konlpy  한국어 

In [ ]:
for coef in coef_neg_index[:50]:
  print(invert_index_vectorizer[coef[1]],coef[0])

# 라쏘

In [ ]:
import mglearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
import mglearn
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

In [ ]:
lasso = Lasso().fit(X_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso.n_iter_))

In [ ]:
#성능 측정

lasso = Lasso().fit(X_train, y_train)
print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso.n_iter_))


lasso001 = Lasso(alpha=0.01, max_iter=100000).fit(X_train, y_train)
print("훈련 세트의 정확도 : {:.2f}".format(lasso001.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso001.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso001.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso001.n_iter_))

lasso00001 = Lasso(alpha=0.0001, max_iter=100000).fit(X_train, y_train)
print("훈련 세트의 정확도 : {:.2f}".format(lasso00001.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso00001.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso00001.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso00001.n_iter_))

In [ ]:
#alpha값에 따른 모델들의 계수 그래프 그리기

# 릿지 회귀
ridge = Ridge().fit(X_train, y_train)
ridge10 = Ridge(alpha=10).fit(X_train, y_train)
ridge01 = Ridge(alpha=0.1).fit(X_train, y_train)

# 라쏘 회귀
lasso = Lasso().fit(X_train, y_train)
lasso001 = Lasso(alpha=0.01, max_iter=100000).fit(X_train, y_train)
lasso00001 = Lasso(alpha=0.0001, max_iter=100000).fit(X_train, y_train)



plt.plot(lasso.coef_, 's', label="Lasso alpha=1")
plt.plot(lasso001.coef_, '^', label="Lasso alpha=0.01")
plt.plot(lasso00001.coef_, 'v', label="Lasso alpha=0.0001")

plt.plot(ridge01.coef_, 'o', label="Ridge alpha=0.1")
plt.legend(ncol=2, loc=(0, 1.05))
plt.xlabel("w list")
plt.ylabel("w size")

# plt.hlines(0, 0, len(lr.coef_))

plt.ylim(-25, 25)
plt.show()

In [ ]:
lasso=Lasso() # LASSO 모형 설정
lasso.fit(X_train, y_train)
#%%
np.sum(lasso.coef_ !=0)

from sklearn.linear_model import LassoCV
lassocv = LassoCV() #Cross Validation을 통해서 lambda값을 설정한다
lassocv.fit(X_train, y_train)
#%%
 
lassocv.alpha_ #Cross Validation을 통해 추출한 lambda값
#%%
lassocv.coef_[:5]
np.sum(lassocv.coef_ !=0)
'''
실제 위에서 추출된 람다값은 0.64정도이며 이에 따라 독립변수의 계수가 0이 될 확률은
더욱 높아졌다. 위의 np.sum의 값은 34개이며, 이는 독립변수의 계수가 0인 아닌것은 것은 총 34개라는
의미
'''
 
#%%
 
mask = lassocv.coef_ != 0
new_reg_data = X_train[:, mask]
new_reg_data.shape

In [ ]:
X, y = mglearn.datasets.load_extended_boston() # 보스턴 주택가격 데이터셋 불러오기

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) # 데이터셋을 훈련셋과 테스트셋으로 분리

lasso = Lasso().fit(X_train, y_train) # 라쏘 회귀 모델 훈련

print("training set score: {:.2f}".format(lasso.score(X_train, y_train))) # 훈련셋 결정계수
print("test set score: {:.2f}".format(lasso.score(X_test, y_test))) # 테스트셋 결정계수
print("the number of the used features: {}".format(np.sum(lasso.coef_ != 0))) # 사용된 특성갯수

usedfeatures = np.nonzero(lasso.coef_ != 0)
print("numbers of the used features: ", usedfeatures[0])# 사용된 특성 번호

In [ ]:
lasso = Lasso(alpha=0.01).fit(X_train, y_train) # 라쏘 회귀 모델 훈련

print("training set score: {:.2f}".format(lasso.score(X_train, y_train))) # 훈련셋 결정계수
print("test set score: {:.2f}".format(lasso.score(X_test, y_test))) # 테스트셋 결정계수
print("the number of the used features: {}".format(np.sum(lasso.coef_ != 0))) # 사용된 특성갯수

usedfeatures = np.nonzero(lasso.coef_ != 0)
print("numbers of the used features: ", usedfeatures[0])# 사용된 특성 번호